In [4]:
from __future__ import annotations

import json
from decimal import Decimal

import pyodbc
from utils import get_secret

class AzureSQLConnection:
    def __init__(self, server, database, username, password, logger, driver='{ODBC Driver 18 for SQL Server}'):
        self.conn_str = f'DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password}'
        self.logger = logger

    def test_connection(self):
        try:
            conn = pyodbc.connect(self.conn_str)
            cursor = conn.cursor()
            cursor.execute('SELECT 1')
            row = cursor.fetchone()
            if row:
                self.logger.info('Connection successful')
            conn.close()
            return True
        except pyodbc.Error as e:
            self.logger.error(f'Connection test failed: {str(e)}')
            return False

    def execute_query(self, query, params=None):
        try:
            conn = pyodbc.connect(self.conn_str)
            cursor = conn.cursor()
            
            if params:
                cursor.execute(query, params)
            else:
                cursor.execute(query)
                
            # For SELECT queries, return results
            if cursor.description:
                # Retrieve column names from the result
                columns = [column[0] for column in cursor.description]
                # Map each row to a dictionary using the column names
                results = [dict(zip(columns, row)) for row in cursor.fetchall()]
                conn.close()
                # Convert Decimals to float if needed
                return json.dumps(results, indent=4, default=lambda o: float(o) if isinstance(o, Decimal) else o)
            else:
                # For non-SELECT queries (INSERT, UPDATE, etc.)
                conn.commit()
                conn.close()
                return True
                
        except pyodbc.Error as e:
            if self.logger:
                self.logger.error(f'Database error: {str(e)}')
            raise Exception(f'Database error: {str(e)}')



In [5]:
import json
import pandas as pd


In [ ]:

DB_SERVER = get_secret('db-server')
DB_NAME = get_secret('db-database')
DB_USER = get_secret('db-username')
DB_PASSWORD = get_secret('db-password')
# 1) instantiate your connection
conn = AzureSQLConnection(
    server=DB_SERVER,
    database=DB_NAME,
    username=DB_USER,
    password=DB_PASSWORD,
    logger=None  # or your logger
)

# 2) define the two CIP13 codes
cip_list = ['3400938014792', '3400938014914', '3400938014624']

# 3) pull only those rows
query = """
SELECT 
    BEN_REG,
    sexe,
    age,
    CIP13,
    PSP_SPE,
    BOITES
FROM dbo.MedicData
WHERE CIP13 IN (?, ?, ?)
"""
# execute and parse JSON into a list of dicts
res_json = conn.execute_query(query, cip_list)
data = json.loads(res_json)

# 4) build DataFrame and ensure BOITES is numeric
df = pd.DataFrame(data)
df['BOITES'] = pd.to_numeric(df['BOITES'], errors='coerce').fillna(0)

# 5) total boxes per (region, sex, age, cip13)
total = (
    df
    .groupby(['BEN_REG', 'sexe', 'age', 'CIP13'], as_index=False)
    .agg(total_boites=('BOITES', 'sum'))
)

# 6) breakdown by prescriber
dist = (
    df
    .groupby(['BEN_REG', 'sexe', 'age', 'CIP13', 'PSP_SPE'], as_index=False)
    .agg(boites_by_prescriber=('BOITES', 'sum'))
)

# pivot so each prescriber becomes its own column
dist_pivot = (
    dist
    .pivot_table(
        index=['BEN_REG', 'sexe', 'age', 'CIP13'],
        columns='PSP_SPE',
        values='boites_by_prescriber',
        fill_value=0
    )
    .reset_index()
)

# 7) merge totals + distribution into one “unified_df”
unified_df = total.merge(dist_pivot, on=['BEN_REG', 'sexe', 'age', 'CIP13'])

# Now `unified_df` has:
#   • BEN_REG, sexe, age, CIP13
#   • total_boites
#   • one column per PSP_SPE giving the box‐count for that prescriber
print(unified_df.head())

    

   BEN_REG  sexe  age          CIP13  total_boites      1     3    4    7  \
0        5     1   20  3400938014792           226  183.0   0.0  0.0  0.0   
1        5     1   20  3400938014914           233  177.0   0.0  0.0  0.0   
2        5     1   60  3400938014792           816  654.0  50.0  0.0  0.0   
3        5     1   60  3400938014914           970  742.0  70.0  0.0  0.0   
4        5     2   20  3400938014792           432  359.0  20.0  0.0  0.0   

     9   14   15   35   42    90    99  
0  0.0  0.0  0.0  0.0  0.0   0.0  43.0  
1  0.0  0.0  0.0  0.0  0.0  26.0  30.0  
2  0.0  0.0  0.0  0.0  0.0  56.0  56.0  
3  0.0  0.0  0.0  0.0  0.0  93.0  65.0  
4  0.0  0.0  0.0  0.0  0.0  25.0  28.0  


In [12]:
unified_df.to_csv('unified_df.csv', index=False)

In [7]:
gen_list = ['815', '814', '813']


# 3) pull only those rows (note WHERE GEN_NUM IN)
query = """
SELECT 
    BEN_REG,
    sexe,
    age,
    GEN_NUM,
    PSP_SPE,
    BOITES
FROM dbo.MedicData
WHERE GEN_NUM IN (?, ?, ?)
"""
res_json = conn.execute_query(query, gen_list)
data = json.loads(res_json)
df = pd.DataFrame(data)

# 4) coerce BOITES to numeric
df['BOITES'] = pd.to_numeric(df['BOITES'], errors='coerce').fillna(0)

# 5) total boxes per (region, sex, age, GEN_NUM)
total = (
    df
    .groupby(['BEN_REG', 'sexe', 'age', 'GEN_NUM'], as_index=False)
    .agg(total_boites=('BOITES', 'sum'))
)

# 6) breakdown by prescriber
dist = (
    df
    .groupby(['BEN_REG', 'sexe', 'age', 'GEN_NUM', 'PSP_SPE'], as_index=False)
    .agg(boites_by_prescriber=('BOITES', 'sum'))
)

# 7) pivot so each prescriber becomes its own column
dist_pivot = (
    dist
    .pivot_table(
        index=['BEN_REG', 'sexe', 'age', 'GEN_NUM'],
        columns='PSP_SPE',
        values='boites_by_prescriber',
        fill_value=0
    )
    .reset_index()
)

# 8) merge totals + distribution into one “unified_df_gen”
unified_df_gen = total.merge(
    dist_pivot,
    on=['BEN_REG', 'sexe', 'age', 'GEN_NUM']
)

# 9) save or inspect
unified_df_gen.to_csv("unified_df_gen.csv", index=False)

In [3]:

def prepare_unified(df: pd.DataFrame, product_type: str) -> pd.DataFrame:
    """
    Standardize brand vs generic DataFrame:
    - Rename product code column to 'product_code'
    - Add 'product_type' column
    """
    df = df.copy()
    # Rename CIP13 or GEN_NUM
    if 'CIP13' in df.columns:
        df = df.rename(columns={'CIP13': 'product_code'})
    elif 'GEN_NUM' in df.columns:
        df = df.rename(columns={'GEN_NUM': 'product_code'})
    else:
        raise KeyError("No product code column found (CIP13 or GEN_NUM)")
    df['product_type'] = product_type
    return df

In [5]:

def compare_brand_vs_generic(brand_df: pd.DataFrame, generic_df: pd.DataFrame) -> dict:
    """
    Compare brand vs generic across segments:
    - Region / Age / Gender segments
    - Prescriber type breakdown
    Returns a dict of DataFrames:
      * 'segment_comparison'
      * 'region_summary'
      * 'age_summary'
      * 'gender_summary'
      * 'prescriber_comparison'
    """
    # Prepare
    brand = prepare_unified(brand_df, 'brand')
    generic = prepare_unified(generic_df, 'generic')

    # === Segment-level (region / sex / age) ===
    brand_seg = brand.groupby(['BEN_REG', 'sexe', 'age'], as_index=False)       .agg(brand_total=('total_boites', 'sum'))
    generic_seg = generic.groupby(['BEN_REG', 'sexe', 'age'], as_index=False)   .agg(generic_total=('total_boites', 'sum'))

    seg = pd.merge(brand_seg, generic_seg, on=['BEN_REG', 'sexe', 'age'], how='outer').fillna(0)
    seg['combined_total'] = seg['brand_total'] + seg['generic_total']
    seg['brand_share'] = seg['brand_total'] / seg['combined_total']
    seg = seg.sort_values('brand_share')

    # Summaries by one dimension:
    region_summary = seg.groupby('BEN_REG', as_index=False).agg(
        brand_total=('brand_total', 'sum'),
        generic_total=('generic_total', 'sum')
    )
    region_summary['combined_total'] = region_summary['brand_total'] + region_summary['generic_total']
    region_summary['brand_share'] = region_summary['brand_total'] / region_summary['combined_total']
    region_summary = region_summary.sort_values('brand_share')

    age_summary = seg.groupby('age', as_index=False).agg(
        brand_total=('brand_total', 'sum'),
        generic_total=('generic_total', 'sum')
    )
    age_summary['combined_total'] = age_summary['brand_total'] + age_summary['generic_total']
    age_summary['brand_share'] = age_summary['brand_total'] / age_summary['combined_total']
    age_summary = age_summary.sort_values('brand_share')

    gender_summary = seg.groupby('sexe', as_index=False).agg(
        brand_total=('brand_total', 'sum'),
        generic_total=('generic_total', 'sum')
    )
    gender_summary['combined_total'] = gender_summary['brand_total'] + gender_summary['generic_total']
    gender_summary['brand_share'] = gender_summary['brand_total'] / gender_summary['combined_total']
    gender_summary = gender_summary.sort_values('brand_share')

    # === Prescriber-level breakdown ===
    prescriber_cols_brand = [c for c in brand.columns if c not in ['BEN_REG', 'sexe', 'age', 'product_code', 'total_boites', 'product_type']]
    prescriber_cols_gen = [c for c in generic.columns if c not in ['BEN_REG', 'sexe', 'age', 'product_code', 'total_boites', 'product_type']]

    brand_presc = brand.melt(
        id_vars=['BEN_REG', 'sexe', 'age'],
        value_vars=prescriber_cols_brand,
        var_name='PSP_SPE',
        value_name='brand_boites'
    )
    generic_presc = generic.melt(
        id_vars=['BEN_REG', 'sexe', 'age'],
        value_vars=prescriber_cols_gen,
        var_name='PSP_SPE',
        value_name='generic_boites'
    )

    presc = pd.merge(
        brand_presc, generic_presc,
        on=['BEN_REG', 'sexe', 'age', 'PSP_SPE'],
        how='outer'
    ).fillna(0)
    presc['combined_total'] = presc['brand_boites'] + presc['generic_boites']
    presc['brand_share'] = presc['brand_boites'] / presc['combined_total']
    presc = presc.sort_values('brand_share')

    return {
        'segment_comparison': seg,
        'region_summary': region_summary,
        'age_summary': age_summary,
        'gender_summary': gender_summary,
        'prescriber_comparison': presc
    }

In [7]:
import pandas as pd

In [8]:
unified_df = pd.read_csv('unified_df.csv')
unified_df_gen = pd.read_csv('unified_df_gen.csv')

metrics = compare_brand_vs_generic(unified_df, unified_df_gen)
# Save outputs
metrics['segment_comparison'].to_csv('segment_comparison_metrics.csv', index=False)
metrics['region_summary'].to_csv('region_summary_metrics.csv', index=False)
metrics['age_summary'].to_csv('age_summary_metrics.csv', index=False)
metrics['gender_summary'].to_csv('gender_summary_metrics.csv', index=False)
metrics['prescriber_comparison'].to_csv('prescriber_comparison_metrics.csv', index=False)
print("All comparison metrics have been saved to CSV files.")


All comparison metrics have been saved to CSV files.


In [11]:
DB_SERVER = get_secret('db-server')
DB_NAME = get_secret('db-database')
DB_USER = get_secret('db-username')
DB_PASSWORD = get_secret('db-password')
# 1) instantiate your connection
conn = AzureSQLConnection(
    server=DB_SERVER,
    database=DB_NAME,
    username=DB_USER,
    password=DB_PASSWORD,
    logger=None  # or your logger
)

In [15]:

# assume conn is your AzureSQLConnection, and gen_list is your list of generic codes
gen_list = ['815', '814']

query = """
SELECT 
    BEN_REG,
    sexe,
    age,
    GEN_NUM,
    CIP13,
    SUM(
        COALESCE(
          TRY_CAST(BOITES AS INT),
          0
        )
    ) AS total_boites
FROM dbo.MedicData
WHERE GEN_NUM IN (?, ?)
GROUP BY
    BEN_REG,
    sexe,
    age,
    GEN_NUM,
    CIP13
"""

# run the query
res_json = conn.execute_query(query, gen_list)
df_gp = pd.DataFrame(json.loads(res_json))

# coerce to numeric
df_gp['total_boites'] = pd.to_numeric(df_gp['total_boites'], errors='coerce').fillna(0).astype(int)

# write out for Streamlit
df_gp.to_csv("generic_products_metrics.csv", index=False)

In [3]:
# Read the unified_df_gen.csv file
df = pd.read_csv("unified_df_gen.csv")

# Calculate total sum of total_boites
total_boxes = df['total_boites'].sum()
print(f"Total sum of total_boites: {total_boxes:,}")

# Calculate total sum of column '42'
total_42 = df['42'].sum()
print(f"Total sum of column '42': {total_42:,}")


Total sum of total_boites: 346,993
Total sum of column '42': 732.0
